In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install lazypredict

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
data = pd.read_csv("/content/drive/MyDrive/features_3_sec.csv")
df = data.sample(frac=1).reset_index(drop=True)
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,pop.00066.4.wav,66149,0.388605,0.084996,0.231507,0.004826,3028.338300,1.174004e+06,3342.191329,100252.036066,...,38.893227,1.093173,104.137634,-2.664141,88.287376,-0.122730,67.860397,2.778727,81.561913,pop
1,jazz.00092.9.wav,66149,0.223066,0.097990,0.107239,0.000987,4273.098132,1.077251e+06,3141.108073,57493.323901,...,55.472553,-6.575345,16.514626,1.380429,42.711636,20.741129,100.786377,-17.499990,33.833736,jazz
2,metal.00032.2.wav,66149,0.477698,0.066282,0.133774,0.000645,2542.632561,2.026325e+05,2190.488081,55577.294906,...,18.485477,-13.634504,34.955769,1.435127,19.866688,-1.927894,20.404568,-0.463197,17.528139,metal
3,metal.00088.2.wav,66149,0.405987,0.092436,0.126362,0.003870,2092.952645,5.325680e+05,2144.801064,103683.963899,...,30.074339,-8.582495,30.162340,-0.980684,26.697878,0.249167,37.676701,-3.475801,21.728697,metal
4,rock.00022.4.wav,66149,0.387875,0.078612,0.180379,0.001631,2191.468959,1.945252e+05,2115.183003,59782.027095,...,46.632908,-5.244864,39.155296,2.295769,27.865902,-5.917521,30.856937,-3.053838,27.441746,rock


In [21]:
dummy = df.drop(['filename', 'length'], axis=1)
column_name = 'label'
label_encoder = LabelEncoder()
dummy[column_name] = label_encoder.fit_transform(dummy[column_name])
corr = dummy.corr()
corr_style = corr.style.background_gradient(cmap='coolwarm')
display(corr_style)

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,perceptr_mean,perceptr_var,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc7_mean,mfcc7_var,mfcc8_mean,mfcc8_var,mfcc9_mean,mfcc9_var,mfcc10_mean,mfcc10_var,mfcc11_mean,mfcc11_var,mfcc12_mean,mfcc12_var,mfcc13_mean,mfcc13_var,mfcc14_mean,mfcc14_var,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
chroma_stft_mean,1.000000,-0.443757,0.424706,0.243647,0.534670,0.251985,0.464617,0.100909,0.526881,0.138394,0.457080,0.278017,0.034305,0.216488,-0.109303,0.417177,0.019084,0.573974,0.054769,-0.513978,0.055816,0.013981,0.121196,0.213522,0.135822,0.069027,0.090913,0.342903,0.121105,0.112848,0.101439,0.423220,0.032791,0.155357,-0.016019,0.376734,-0.067534,0.108459,-0.147130,0.400529,-0.226017,-0.036171,-0.242423,0.297993,-0.265843,-0.078784,-0.270092,0.228293,-0.264815,-0.103968,-0.273841,0.168254,-0.308342,-0.062985,-0.313233,0.193666,-0.363354,0.330370
chroma_stft_var,-0.443757,1.000000,-0.078410,0.251900,-0.208136,0.276964,-0.031197,0.291007,-0.141792,0.330930,-0.391281,0.138214,-0.010949,0.024139,0.072719,0.000478,-0.004419,-0.296517,0.347460,0.091757,0.326217,0.323529,0.354501,-0.194552,0.333186,0.248247,0.351826,-0.155817,0.363150,0.192922,0.343599,-0.239743,0.352686,0.126653,0.363827,-0.216207,0.337400,0.149261,0.335556,-0.263772,0.302924,0.209318,0.309993,-0.197412,0.283260,0.162830,0.285569,-0.221620,0.296099,0.111544,0.303262,-0.171744,0.295359,0.107008,0.277295,-0.173398,0.287195,-0.079683
rms_mean,0.424706,-0.078410,1.000000,0.553770,0.470781,0.241796,0.495142,0.085760,0.500178,0.157281,0.293745,0.217606,0.065134,0.884846,-0.012401,0.766446,0.015668,0.795000,-0.080320,-0.453606,0.033635,0.136936,0.069158,-0.034473,0.178296,0.122941,0.119419,0.072916,0.212010,0.181207,0.140892,0.134868,0.126329,0.213195,0.087181,0.144650,0.070977,0.133593,-0.012006,0.134779,-0.064247,0.043769,-0.088406,0.113960,-0.080352,0.012184,-0.088842,0.055563,-0.032953,-0.013952,-0.041470,0.087615,-0.046180,0.024552,-0.062662,0.082251,-0.103519,0.205471
rms_var,0.243647,0.251900,0.553770,1.000000,0.327809,0.509235,0.383329,0.285950,0.350716,0.377474,0.143768,0.453957,0.071193,0.519717,0.100449,0.744850,-0.020418,0.296198,0.336492,-0.351508,0.355515,0.277005,0.376544,-0.215820,0.412464,0.248711,0.359285,-0.134974,0.479166,0.290245,0.356301,-0.110396,0.381793,0.301416,0.332587,-0.058046,0.297938,0.251421,0.227644,-0.066374,0.164785,0.214514,0.161973,-0.026530,0.130442,0.189566,0.116201,-0.064570,0.159390,0.144238,0.144163,-0.004785,0.102019,0.136798,0.095054,0.015085,0.073571,0.101467
spectral_centroid_mean,0.534670,-0.208136,0.470781,0.327809,1.000000,0.476959,0.890382,0.021120,0.974360,0.172380,0.865487,0.579997,0.050559,0.274194,-0.004654,0.531487,0.002111,0.686196,-0.061331,-0.931435,0.085022,0.195977,0.042125,-0.165793,0.187926,0.078506,0.027291,-0.027122,0.209412,0.196388,0.106182,0.088172,0.118458,0.260035,0.057883,0.146238,0.054746,0.144027,-0.009416,0.118541,-0.064614,0.058537,-0.055983,0.118186,-0.058378,0.035098,-0.064673,0.116866,-0.025006,0.026879,-0.018358,0.177934,-0.006376,0.018651,-0.011018,0.191512,-0.051205,0.360175
spectral_centroid_var,0.251985,0.276964,0.241796,0.509235,0.476959,1.000000,0.556491,0.614254,0.492965,0.780308,0.242913,0.818348,0.047028,0.192711,0.089544,0.388760,-0.009407,0.159586,0.429485,-0.497158,0.748612,0.513633,0.461658,-0.297699,0.488902,0.406538,0.390078,-0.158945,0.493938,0.391931,0.403324,-0.158683,0.405858,0.374794,0.357289,-0.135941,0.326240,0.309186,0.244125,-0.139430,0.160181,0.282591,0.166777,-0.113106,0.111909,0.234941,0.096053,-0.134131,0.125906,0.134869,0.115752,-0.045517,0.093684,0.12385

In [7]:
label_encoder = LabelEncoder()
target = "label"
x = df.drop(['filename', 'length',target], axis=1)
y = df[target]
y= label_encoder.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=100)
y

array([7, 3, 5, ..., 0, 0, 3])

In [8]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train

array([[ 0.25879486,  0.34771182,  1.80544582, ..., -0.85760257,
        -0.57065063, -0.68390099],
       [-0.22844685,  1.12587602, -0.01774581, ...,  0.25714037,
         0.68534788,  0.00660484],
       [-0.10127454,  1.18134052, -0.46575897, ...,  0.09685155,
        -0.36041918,  0.89591963],
       ...,
       [-0.32427007,  0.72952411,  1.45828582, ...,  0.37975343,
         0.85090947, -0.20560639],
       [-0.73895391,  0.6790055 ,  0.48712168, ..., -0.6292927 ,
        -0.47660985, -0.64529929],
       [ 0.84781952, -0.14029105,  1.19552582, ..., -0.79384673,
         1.9488413 , -0.63914817]])

In [9]:
model = LGBMClassifier(boosting_type= 'gbdt', learning_rate=0.2, max_depth= 7, num_leaves=150)
model.fit(x_train,y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009440 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14329
[LightGBM] [Info] Number of data points in the train set: 7992, number of used features: 57
[LightGBM] [Info] Start training from score -2.296597
[LightGBM] [Info] Start training from score -2.305342
[LightGBM] [Info] Start training from score -2.285465
[LightGBM] [Info] Start training from score -2.291634
[LightGBM] [Info] Start training from score -2.274456
[LightGBM] [Info] Start training from score -2.304088
[LightGBM] [Info] Start training from score -2.312898
[LightGBM] [Info] Start training from score -2.346320
[LightGBM] [Info] Start training from score -2.286696
[LightGBM] [Info] Start training from score -2.324342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

LGBMClassifier(learning_rate=0.2, max_depth=7, num_leaves=150)

In [10]:
predictions = model.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))
print(classification_report(y_test, predictions))

Accuracy: 91.49%
              precision    recall  f1-score   support

           0       0.93      0.89      0.91       196
           1       0.92      0.97      0.94       201
           2       0.84      0.89      0.87       184
           3       0.88      0.91      0.89       191
           4       0.95      0.89      0.92       176
           5       0.93      0.92      0.92       202
           6       0.95      0.96      0.95       209
           7       0.94      0.94      0.94       235
           8       0.93      0.89      0.91       188
           9       0.88      0.89      0.88       216

    accuracy                           0.91      1998
   macro avg       0.92      0.91      0.91      1998
weighted avg       0.92      0.91      0.92      1998



In [27]:
cm=confusion_matrix(y_test, predictions)
category_names = ["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]
true_positives = np.diag(cm)
false_positives = np.sum(cm, axis=0) - true_positives
false_negatives = np.sum(cm, axis=1) - true_positives
accuracy_per_label = true_positives / (true_positives + false_negatives)
print(confusion_matrix(y_test, predictions))
for i, accuracy in enumerate(accuracy_per_label):
    category_name = category_names[i]
    print("Accuracy for {}: {:.4f}".format(category_name, accuracy))

[[175   1   7   2   2   4   2   0   2   1]
 [  0 194   1   0   0   4   0   0   0   2]
 [  6   1 164   3   0   3   0   1   1   5]
 [  1   0   5 173   0   1   0   4   3   4]
 [  1   0   2   3 157   0   3   6   1   3]
 [  2  10   3   0   0 185   0   0   0   2]
 [  1   0   0   2   1   0 200   0   0   5]
 [  0   1   1   7   0   1   0 221   3   1]
 [  0   2   6   1   4   0   1   4 167   3]
 [  3   1   6   5   1   2   4   0   2 192]]
Accuracy for blues: 0.8929
Accuracy for classical: 0.9652
Accuracy for country: 0.8913
Accuracy for disco: 0.9058
Accuracy for hiphop: 0.8920
Accuracy for jazz: 0.9158
Accuracy for metal: 0.9569
Accuracy for pop: 0.9404
Accuracy for reggae: 0.8883
Accuracy for rock: 0.8889
